# Start

In [6]:
import importlib
import os
import sys
import csv
import datetime 
import json
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
from pathlib import Path
from docx import Document
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.shared import Inches
from typing import Optional

from langchain_core.tools import Tool, StructuredTool
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage

from bsm_multi_agents.graph.state import WorkflowState

In [7]:
cwd = Path.cwd()
project_path = cwd.parent.parent
project_path

PosixPath('/Users/yifanli/Github/model_doc_automation/TooTwo_mcp')

# MCP Server

## call_mcp_tool_async

In [3]:
from bsm_multi_agents.agents.mcp_adapter import call_mcp_tool

In [9]:
server_script_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
tool_name = "calculate_bsm_to_file"
args = {
    "input_path": csv_file_path,
    "output_dir": output_dir,
}

In [10]:
call_mcp_tool(tool_name, server_script_path, args)

CallToolResult(meta=None, content=[TextContent(type='text', text='Unknown tool: calculate_bsm_to_file', annotations=None, meta=None)], structuredContent=None, isError=True)

## list_mcp_tools_sync

In [4]:
from bsm_multi_agents.agents.mcp_adapter import list_mcp_tools_sync

In [5]:
server_script_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")

In [6]:
list_mcp_tools_sync(server_script_path)

[Tool(name='calculate_greeks_to_file', title=None, description='\n    Reads the CSV at input_path, calculates greeks, and saves the result to output_dir.\n    Returns the path to the result file.\n    ', inputSchema={'properties': {'input_path': {'title': 'Input Path', 'type': 'string'}, 'output_dir': {'default': './output', 'title': 'Output Dir', 'type': 'string'}}, 'required': ['input_path'], 'title': 'calculate_greeks_to_fileArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': 'calculate_greeks_to_fileOutput', 'type': 'object'}, icons=None, annotations=None, meta=None),
 Tool(name='validate_greeks_to_file', title=None, description='\n    Validate Greeks for ALL options from CSV data.\n\n    For each option:\n    - Validates: price > 0\n    - Validates: delta in [0,1] for calls, [-1,0] for puts\n    - Validates: gamma >= 0, vega >= 0\n\n    Args:\n        state: InjectedState, state from the wor

## mcp_tool_to_langchain_tool

In [7]:
from bsm_multi_agents.agents.mcp_adapter import list_mcp_tools_sync
server_script_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
mcp_tools = list_mcp_tools_sync(server_script_path)
mcp_tool = mcp_tools[0]
print(mcp_tool)

name='calculate_greeks_to_file' title=None description='\n    Reads the CSV at input_path, calculates greeks, and saves the result to output_dir.\n    Returns the path to the result file.\n    ' inputSchema={'properties': {'input_path': {'title': 'Input Path', 'type': 'string'}, 'output_dir': {'default': './output', 'title': 'Output Dir', 'type': 'string'}}, 'required': ['input_path'], 'title': 'calculate_greeks_to_fileArguments', 'type': 'object'} outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': 'calculate_greeks_to_fileOutput', 'type': 'object'} icons=None annotations=None meta=None


In [8]:
from bsm_multi_agents.agents.mcp_adapter import mcp_tool_to_langchain_tool

In [9]:
test_tool = mcp_tool_to_langchain_tool(mcp_tool, server_script_path)

In [10]:
test_tool.invoke({"input_path": csv_file_path, "output_dir": output_dir})

CallToolResult(meta=None, content=[TextContent(type='text', text='/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results.csv', annotations=None, meta=None)], structuredContent={'result': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results.csv'}, isError=False)

# Tools

## calculate_bsm_to_file

In [17]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [18]:
from bsm_multi_agents.mcp import pricing_calculator
importlib.reload(pricing_calculator)
from bsm_multi_agents.mcp.pricing_calculator import calculate_bsm_to_file

In [19]:
calculate_bsm_to_file(csv_file_path, output_dir)

/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/pricing_calculator.py:11: RuntimeWarning: invalid value encountered in sqrt
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/pricing_calculator.py:12: RuntimeWarning: invalid value encountered in sqrt
  d2 = d1 - sigma * np.sqrt(T)
/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/pricing_calculator.py:11: RuntimeWarning: invalid value encountered in log
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))


'/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv'

## calculate_greeks_to_file

In [20]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [21]:
from bsm_multi_agents.mcp import pricing_calculator
importlib.reload(pricing_calculator)
from bsm_multi_agents.mcp.pricing_calculator import calculate_greeks_to_file

In [22]:
calculate_greeks_to_file(csv_file_path, output_dir)

'/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results.csv'

## validate_greeks_to_file

In [106]:
greeks_results_path = os.path.join(project_path, "data/cache/dummy_options_greeks_results.csv")
output_dir = os.path.join(project_path, "data/cache")

In [107]:
from bsm_multi_agents.mcp import pricing_validator
importlib.reload(pricing_validator)
from bsm_multi_agents.mcp.pricing_validator import validate_greeks_to_file

In [108]:
validate_greeks_to_file(greeks_results_path, output_dir)

'/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results_validate_results.csv'

## run_sensitivity_test_to_file

In [3]:
input_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [4]:
df = pd.read_csv(input_path)
required_cols = ['option_type', 'S', 'K', 'T', 'r', 'sigma']
missing = [c for c in required_cols if c not in df.columns]
missing

[]

In [5]:
from bsm_multi_agents.mcp.pricing_validator import _run_sens

In [8]:
row = df.iloc[0].to_dict()
option_json = json.dumps({
    "option_type": row.get('option_type'),
    "S": float(row.get('S')),
    "K": float(row.get('K')),
    "T": float(row.get('T')),
    "r": float(row.get('r', 0.0)),
    "sigma": float(row.get('sigma'))
})

In [ ]:
test_result = _run_sens(option_json, output_dir=output_dir)

'/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_summary.docx'

In [ ]:
doc.save(output_dir)

## write_report_to_word

# Agent

## pricing_calculator_agent_node

### Inside Node

In [14]:
from bsm_multi_agents.config.llm_config import get_llm
from bsm_multi_agents.agents.utils import extract_mcp_content, load_tools_from_mcp_and_local


In [17]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
local_tool_paths = [os.path.join(project_path, "src/bsm_multi_agents/tools/my_add.py")]

In [18]:
state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path,
    local_tool_paths=local_tool_paths
)

In [19]:
errors = state.get("errors", [])
"csv_file_path" not in state or not state["csv_file_path"]
server_path = state.get("server_path")
output_dir = state.get("output_dir")

In [20]:
local_tool_paths = state.get("local_tool_paths", [])
langchain_tools = load_tools_from_mcp_and_local(server_path, local_tool_paths)
langchain_tools

[StructuredTool(name='calculate_greeks_to_file', description='Reads the CSV at input_path, calculates greeks, and saves the result to output_dir.\n    Returns the path to the result file.', args_schema=<class 'bsm_multi_agents.agents.mcp_adapter.calculate_greeks_to_fileInput'>, func=<function mcp_tool_to_langchain_tool.<locals>.tool_func at 0x119749260>),
 StructuredTool(name='validate_greeks_to_file', description='Validate Greeks for ALL options from CSV data.\n\n    For each option:\n    - Validates: price > 0\n    - Validates: delta in [0,1] for calls, [-1,0] for puts\n    - Validates: gamma >= 0, vega >= 0\n\n    Args:\n        state: InjectedState, state from the workflow, which contains csv_data\n\n\n    Returns:\n        JSON string containing validate_results', args_schema=<class 'bsm_multi_agents.agents.mcp_adapter.validate_greeks_to_fileInput'>, func=<function mcp_tool_to_langchain_tool.<locals>.tool_func at 0x118597e20>),
 StructuredTool(name='run_sensitivity_test_to_file', 

In [21]:
llm = get_llm().bind_tools(langchain_tools)

In [22]:
system_prompt = (
    "You are a quantitative calculator agent. "
    "You have access to tools specifically for Greeks calculation via an MCP server. "
    "Use the available tools to process the requested data. "
    "If you are confident, you can run all tools in parallel."
)

user_prompt = (
    f"Input CSV File: {state['csv_file_path']}\n"
    f"Output Directory: {output_dir}\n\n"
    "Please calculate the Greeks for the options in the input CSV file. "
    "Save the results to the output directory. "
    "Ensure you call the calculation tools."
)

messages = list(state.get("messages", []))
if not messages:
        messages.append(SystemMessage(content=system_prompt))
        messages.append(HumanMessage(content=user_prompt))

In [23]:
ai_msg = llm.invoke(messages)
messages.append(ai_msg)
state["messages"] = messages

In [24]:
ai_msg.tool_calls

[{'name': 'calculate_greeks_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache'},
  'id': 'bbe34e6d-197d-44b6-b33a-4f28ed11d1d5',
  'type': 'tool_call'}]

### Node Level

In [25]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [26]:
from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import pricing_calculator_agent_node

In [27]:
initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

In [28]:
state = pricing_calculator_agent_node(initial_state)

In [29]:
state['messages'][-1].tool_calls

[{'name': 'calculate_greeks_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache'},
  'id': '091e0859-4d9b-43cd-b0d9-6f5de14edf10',
  'type': 'tool_call'}]

## pricing_calculator_tool_node

### Inside Node

In [ ]:
from bsm_multi_agents.config.llm_config import get_llm
from bsm_multi_agents.agents.mcp_adapter import list_mcp_tools_sync

In [ ]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
local_tool_paths = [os.path.join(project_path, "src/bsm_multi_agents/tools/my_add.py")]

from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import pricing_calculator_agent_node

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path,
    local_tool_paths=local_tool_paths
)

state = pricing_calculator_agent_node(initial_state)

In [ ]:
from bsm_multi_agents.agents.mcp_adapter import call_mcp_tool
from bsm_multi_agents.agents.utils import extract_mcp_content

In [82]:
errors = state.get("errors", [])
messages = list(state.get("messages", []))

In [83]:
last_msg = messages[-1]
not hasattr(last_msg, "tool_calls") or not last_msg.tool_calls

False

In [84]:
server_path = state.get("server_path")

In [85]:
"tool_outputs" not in state or state["tool_outputs"] is None

True

In [86]:
tool_outputs_msgs = []

In [90]:
last_msg.tool_calls

[{'name': 'calculate_greeks_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache'},
  'id': '535b3148-714e-4e8d-9641-31b12403dc34',
  'type': 'tool_call'}]

In [ ]:
tool_call = last_msg.tool_calls[0]
tool_name = tool_call["name"]
args = tool_call["args"]
call_id = tool_call["id"]

In [ ]:
raw_result = call_mcp_tool(tool_name, server_path, args)
result_text = extract_mcp_content(raw_result)
result_text

In [92]:
tool_outputs_msgs.append(ToolMessage(content=result_text, tool_call_id=call_id, name=tool_name))
state["greeks_results_path"] = result_text.strip()

In [56]:
content = getattr(tool_res, "content", None)

In [60]:
content[0].text

'/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv'

### Node Level

In [36]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

state = pricing_calculator_agent_node(initial_state)
state = pricing_calculator_tool_node(state)

In [37]:
state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/mcp/server.py',
 'messages': [SystemMessage(content='You are a quantitative calculator agent. You have access to tools specifically for Greeks calculation via an MCP server. Use the available tools to process the requested data. If you are confident, you can run all tools in parallel.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Input CSV File: /Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv\nOutput Directory: /Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache\n\nPlease calculate the Greeks for the options in the input CSV file. Save the results to the output directory. Ensure you call the calculation tools.', additional_kwargs={}, respon

## pricing_validator_agent_node

### Inside Node

In [45]:
from bsm_multi_agents.config.llm_config import get_llm
from bsm_multi_agents.agents.mcp_adapter import call_mcp_tool
from bsm_multi_agents.agents.utils import load_tools_from_mcp_and_local

In [46]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
local_tool_paths = [os.path.join(project_path, "src/bsm_multi_agents/tools/my_add.py")]

from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path,
    local_tool_paths=local_tool_paths
)

state = pricing_calculator_agent_node(initial_state)
state = pricing_calculator_tool_node(initial_state)

In [47]:
errors = state.get("errors", [])
"greeks_results_path" not in state or not state["greeks_results_path"]

False

In [48]:
server_path = state.get("server_path")
output_dir = state.get("output_dir")

In [49]:
local_tool_paths = state.get("local_tool_paths", [])
langchain_tools = load_tools_from_mcp_and_local(server_path, local_tool_paths)
langchain_tools

[StructuredTool(name='calculate_greeks_to_file', description='Reads the CSV at input_path, calculates greeks, and saves the result to output_dir.\n    Returns the path to the result file.', args_schema=<class 'bsm_multi_agents.agents.mcp_adapter.calculate_greeks_to_fileInput'>, func=<function mcp_tool_to_langchain_tool.<locals>.tool_func at 0x11b0be5c0>),
 StructuredTool(name='validate_greeks_to_file', description='Validate Greeks for ALL options from CSV data.\n\n    For each option:\n    - Validates: price > 0\n    - Validates: delta in [0,1] for calls, [-1,0] for puts\n    - Validates: gamma >= 0, vega >= 0\n\n    Args:\n        state: InjectedState, state from the workflow, which contains csv_data\n\n\n    Returns:\n        JSON string containing validate_results', args_schema=<class 'bsm_multi_agents.agents.mcp_adapter.validate_greeks_to_fileInput'>, func=<function mcp_tool_to_langchain_tool.<locals>.tool_func at 0x11b125760>),
 StructuredTool(name='run_sensitivity_test_to_file', 

In [50]:
llm = get_llm().bind_tools(langchain_tools)

In [51]:
system_prompt = (
    "You are a quantitative validator agent. "
    "You have access to tools specifically for validation of greeks via an MCP server. "
    "Use the available tools to process the requested data. "
)

user_prompt = (
    f"Input CSV File: {state['greeks_results_path']}\n"
    f"Output Directory: {output_dir}\n\n"
    "Please validate the Greeks for the options in the input CSV file. "
    "Save the results to the output directory. "
    "Ensure you call the validation tools."
)
messages = list(state.get("messages", []))
messages.append(SystemMessage(content=system_prompt))
messages.append(HumanMessage(content=user_prompt))


In [52]:
ai_msg = llm.invoke(messages)

In [53]:
ai_msg.tool_calls

[{'name': 'validate_greeks_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache'},
  'id': '34e6b4de-b6a7-4b5c-9422-dc08d2c1f2a9',
  'type': 'tool_call'}]

### Node Level

In [54]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
local_tool_paths = [os.path.join(project_path, "src/bsm_multi_agents/tools/my_add.py")]

from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)


from bsm_multi_agents.agents.pricing_validator_node import (
    pricing_validator_agent_node,
)

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path,
    local_tool_paths=local_tool_paths
)

state = pricing_calculator_agent_node(initial_state)
state = pricing_calculator_tool_node(state)
state = pricing_validator_agent_node(state)
state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/mcp/server.py',
 'local_tool_paths': ['/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/tools/my_add.py'],
 'messages': [SystemMessage(content='You are a quantitative calculator agent. You have access to tools specifically for Greeks calculation via an MCP server. Use the available tools to process the requested data. If you are confident, you can run all tools in parallel.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Input CSV File: /Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv\nOutput Directory: /Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache\n\nPlease calculate the Greeks for the options in the input CSV 

In [55]:
state['messages'][-1].tool_calls

[{'name': 'validate_greeks_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache'},
  'id': 'ef7c7108-16af-4eb6-8606-dc92dcd0f00d',
  'type': 'tool_call'}]

## pricing_validator_tool_node

### Inside Node

In [56]:
from bsm_multi_agents.agents.mcp_adapter import call_mcp_tool
from bsm_multi_agents.agents.utils import extract_mcp_content

In [ ]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
local_tool_paths = [os.path.join(project_path, "src/bsm_multi_agents/tools/my_add.py")]

from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)


from bsm_multi_agents.agents.pricing_validator_node import (
    pricing_validator_agent_node,
)

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path,
    local_tool_paths=local_tool_paths
)

state = pricing_calculator_agent_node(initial_state)
state = pricing_calculator_tool_node(state)
state = pricing_validator_agent_node(state)
state

In [59]:
errors = state.get("errors", [])
messages = list(state.get("messages", []))
last_msg = messages[-1]
server_path = state.get("server_path")
tool_outputs_msgs = []
    
for tool_call in last_msg.tool_calls:
    tool_name = tool_call["name"]
    args = tool_call["args"]
    call_id = tool_call["id"]
    
    try:
        # Execute logic
        raw_result = call_mcp_tool(tool_name, server_path, args)
        result_text = extract_mcp_content(raw_result)
        
        # Create ToolMessage
        tool_outputs_msgs.append(ToolMessage(content=result_text, tool_call_id=call_id, name=tool_name))
        
        # Generic Output Handling
        state["validate_results_path"] = result_text.strip()

    except Exception as e:
        err_msg = f"Error executing {tool_name}: {e}"
        errors.append(err_msg)
        tool_outputs_msgs.append(ToolMessage(content=err_msg, tool_call_id=call_id, is_error=True))
            

In [60]:
tool_outputs_msgs

[ToolMessage(content='/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results_validate_results.csv', name='validate_greeks_to_file', tool_call_id='ef7c7108-16af-4eb6-8606-dc92dcd0f00d')]

### Node Level

In [61]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
local_tool_paths = [os.path.join(project_path, "src/bsm_multi_agents/tools/my_add.py")]

from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)


from bsm_multi_agents.agents.pricing_validator_node import (
    pricing_validator_agent_node,
    pricing_validator_tool_node,
)

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path,
    local_tool_paths=local_tool_paths
)

state = pricing_calculator_agent_node(initial_state)
state = pricing_calculator_tool_node(state)
state = pricing_validator_agent_node(state)
state = pricing_validator_tool_node(state)
state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/mcp/server.py',
 'local_tool_paths': ['/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/tools/my_add.py'],
 'messages': [SystemMessage(content='You are a quantitative calculator agent. You have access to tools specifically for Greeks calculation via an MCP server. Use the available tools to process the requested data. If you are confident, you can run all tools in parallel.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Input CSV File: /Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv\nOutput Directory: /Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache\n\nPlease calculate the Greeks for the options in the input CSV 

## report_generator_node

### Inside Node

In [121]:
from bsm_multi_agents.config.llm_config import get_llm

In [122]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
local_tool_paths = [os.path.join(project_path, "src/bsm_multi_agents/tools/my_add.py")]
final_report_path = os.path.join(project_path, "data/output/final_report.docx")

from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)


from bsm_multi_agents.agents.pricing_validator_node import (
    pricing_validator_agent_node,
    pricing_validator_tool_node,
)

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path,
    local_tool_paths=local_tool_paths,
    final_report_path=final_report_path,
)

state = pricing_calculator_agent_node(initial_state)
state = pricing_calculator_tool_node(state)
state = pricing_validator_agent_node(state)
state = pricing_validator_tool_node(state)
state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/mcp/server.py',
 'local_tool_paths': ['/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/tools/my_add.py'],
 'final_report_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/output/final_report.docx',
 'messages': [SystemMessage(content='You are a quantitative calculator agent. You have access to tools specifically for Greeks calculation via an MCP server. Use the available tools to process the requested data. If you are confident, you can run all tools in parallel.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Input CSV File: /Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv\nOutput Directory: /Users/yifanli/Githu

In [123]:
errors = state.get("errors", [])
csv_path = state.get("validate_results_path")
output_dir = state.get("output_dir")
validate_results_path = state.get("validate_results_path")
final_report_path = state.get("final_report_path")


In [124]:
title: str = "Ongoing Monitoring Analysis Report"
model_name: str = "Option Pricing, BSM"
author_name: str = "John Doe"
group_name: str = "Front Desk Modeling and Analytics"
version: str = "v1.0"
section1_heading: str = "1. Introduction"
section1_paragraph: Optional[str] = None
section2_heading: str = "2. Summary of Analysis"

In [125]:
llm = get_llm()

In [126]:
doc = Document()
# Title
title_run = doc.add_paragraph().add_run(title)
title_run.bold = True
title_run.font.size = doc.styles["Title"].font.size
doc.paragraphs[-1].alignment = 1  # 0=left, 1=center

# Subtitle (Model Name)
subtitle_para = doc.add_paragraph()
subtitle_run = subtitle_para.add_run(model_name)
subtitle_para.alignment = 1
subtitle_run.bold = True

doc.add_paragraph("")  # spacer

# Author
author_para = doc.add_paragraph()
author_para.add_run("Author: ").bold = True
author_para.add_run(author_name).italic = False
author_para.alignment = 1

# Group
group_para = doc.add_paragraph()
group_para.add_run("Group: ").bold = True
group_para.add_run(group_name)
group_para.alignment = 1

# Report date
date_str = datetime.date.today().strftime("%B %d, %Y")
date_para = doc.add_paragraph()
date_para.add_run("Report Date: ").bold = True
date_para.add_run(date_str)
date_para.alignment = 1

# Version
version_para = doc.add_paragraph()
version_para.add_run("Document Version: ").bold = True
version_para.add_run(version)
version_para.alignment = 1

# Page break after title page
doc.add_page_break()

# ============================
# TABLE OF CONTENTS
# ============================
doc.add_heading("Table of Contents", level=1)

toc_para = doc.add_paragraph()
fld = OxmlElement("w:fldSimple")
fld.set(qn("w:instr"), 'TOC \\o "1-3" \\h \\z \\u')
toc_para._p.append(fld)

doc.add_page_break()

# ============================
# SECTION 1
# ============================
if not section1_paragraph:
    section1_paragraph = (
        "This section provides contextual background, objectives, and relevant "
        "considerations for the ongoing monitoring analysis. Subsequent sections "
        "expand on methodology, insights, and results."
    )

doc.add_heading(section1_heading, level=1)
doc.add_paragraph(section1_paragraph)

# ============================
# SECTION 2 – Refined Summary
# ============================
doc.add_heading(section2_heading, level=1)



# Read CSV
# with open(pricing_path, newline="", encoding="utf-8") as f:
#     reader = csv.reader(f)
#     rows = list(reader)
df_validate = pd.read_csv(validate_results_path)

for asset in ["FX", "Equity", "Commodity"]:
    doc.add_paragraph("The pricing output of "+asset+ " listed in the below table,")
    df = df_validate[df_validate["asset_class"] == asset]
    df = df.sort_values("T")
    # df = df.dropna()
    
    fig, ax = plt.subplots()
    df_call = df[df["option_type"] == "call"]
    ax.plot(df_call["T"], df_call["BSM_price"], label = "call")
    df_put = df[df["option_type"] == "put"]
    ax.plot(df_put["T"], df_put["BSM_price"], label = "put")

    ax.set_xlabel("Time to Maturity (T)")
    ax.set_ylabel("Option Price (BSM)")
    ax.set_title(f"Option Pricing Curve – {asset}")  
    ax.legend()

    # Save figure to memory (PNG bytes)
    img_stream = BytesIO()
    fig.savefig(img_stream, format="png", dpi=200, bbox_inches="tight")
    plt.close(fig)
    img_stream.seek(0)

    system_prompt = (
        "Please summmarize the option pricing results (pull and call) from the tables with the following topics"
        "1. Overall Data Quality"
        "2. Pricing Level by Asset Class"
        "3. Term Structure (Price vs Maturity)"
        "4. Call vs Put Behavior"
        "5. Model Consistency"
        "6. Key Takeaway"
        "Also, we have some annotation for you to understand table columns"
        "- **Valuation Date:** The date on which the option price is calculated."
        "- **Spot Price (S):** Current price of the underlying asset."
        "- **Strike Price (K):** Exercise price of the option."
        "- **Time to Maturity (T):** Time remaining until option expiration, expressed in years."
        "- **Risk-Free Rate (r):** Annualized risk-free interest rate, used for discounting."
        "- **Volatility (σ):** Annualized standard deviation of the underlying asset’s returns."
        "- **Option Type:** Call or Put."
        "- **Asset Class:** Classification of the underlying asset (e.g., equity, index)."
    )

    user_prompt = (
        "Here is the raw pull and call price tables that should become the option pricing output summary section. The title should have this asset class name."
        "Please refine it as described:\n\n"
        f"{df_call}"
        f"{df_put}"
    )

    # # Using chat.completions; you can swap to Responses API if you prefer
    # completion = client.chat.completions.create(
    #     model="gpt-4.1-mini",
    #     messages=[
    #         {"role": "system", "content": system_prompt},
    #         {"role": "user", "content": user_prompt},
    #     ],
    #     temperature=0.3,
    # )
            

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_prompt),
    ]
    ai_msg = llm.invoke(messages)
    refined_summary1 = ai_msg.content
    
    for block in refined_summary1.split("\n\n"):
        block = block.strip()
        if block:
            doc.add_paragraph(block)   

    # Insert figure
    doc.add_picture(img_stream, width=Inches(6.5))

    # Create table (rows = data + header)
    table = doc.add_table(rows=df.shape[0] + 1, cols=df.shape[1])
    table.style = "Table Grid"

    # Header row
    for col_idx, col_name in enumerate(df.columns):
        cell = table.rows[0].cells[col_idx]
        cell.text = str(col_name)
        cell.paragraphs[0].runs[0].bold = True

    # Data rows
    for row_idx in range(df.shape[0]):
        for col_idx in range(df.shape[1]):
            table.rows[row_idx + 1].cells[col_idx].text = str(df.iat[row_idx, col_idx])


In [127]:
doc.save(final_report_path)

In [95]:
asset="Equity"
doc.add_paragraph("The pricing output of "+asset+ " listed in the below table,")
df = df_validate[df_validate["asset_class"] == asset]
df = df.sort_values("T")
# df = df.dropna()


In [106]:
fig, ax = plt.subplots()
df_call = df[df["option_type"] == "call"]
ax.plot(df_call["T"], df_call["BSM_price"], label = "call")
df_put = df[df["option_type"] == "put"]
ax.plot(df_put["T"], df_put["BSM_price"], label = "put")

ax.set_xlabel("Time to Maturity (T)")
ax.set_ylabel("Option Price (BSM)")
ax.set_title(f"Option Pricing Curve – {asset}")  
ax.legend()

# Save figure to memory (PNG bytes)
img_stream = BytesIO()
fig.savefig(img_stream, format="png", dpi=200, bbox_inches="tight")
plt.close(fig)
img_stream.seek(0)

0

In [107]:
system_prompt = (
    "Please summmarize the option pricing results (pull and call) from the tables with the following topics"
    "1. Overall Data Quality"
    "2. Pricing Level by Asset Class"
    "3. Term Structure (Price vs Maturity)"
    "4. Call vs Put Behavior"
    "5. Model Consistency"
    "6. Key Takeaway"
    "Also, we have some annotation for you to understand table columns"
    "- **Valuation Date:** The date on which the option price is calculated."
    "- **Spot Price (S):** Current price of the underlying asset."
    "- **Strike Price (K):** Exercise price of the option."
    "- **Time to Maturity (T):** Time remaining until option expiration, expressed in years."
    "- **Risk-Free Rate (r):** Annualized risk-free interest rate, used for discounting."
    "- **Volatility (σ):** Annualized standard deviation of the underlying asset’s returns."
    "- **Option Type:** Call or Put."
    "- **Asset Class:** Classification of the underlying asset (e.g., equity, index)."
)

user_prompt = (
    "Here is the raw pull and call price tables that should become the option pricing output summary section. The title should have this asset class name."
    "Please refine it as described:\n\n"
    f"{df_call}"
    f"{df_put}"
)

In [115]:
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]
ai_msg = llm.invoke(messages)
refined_summary1 = ai_msg.content

In [116]:
for block in refined_summary1.split("\n\n"):
    block = block.strip()
    if block:
        doc.add_paragraph(block)   

In [120]:
doc.add_picture(img_stream, width=Inches(6.5))
table = doc.add_table(rows=df.shape[0] + 1, cols=df.shape[1])
table.style = "Table Grid"
for col_idx, col_name in enumerate(df.columns):
        cell = table.rows[0].cells[col_idx]
        cell.text = str(col_name)
        cell.paragraphs[0].runs[0].bold = True

# Data rows
for row_idx in range(df.shape[0]):
    for col_idx in range(df.shape[1]):
        table.rows[row_idx + 1].cells[col_idx].text = str(df.iat[row_idx, col_idx])


### Node Level

In [3]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
local_tool_paths = [os.path.join(project_path, "src/bsm_multi_agents/tools/my_add.py")]
final_report_path = os.path.join(project_path, "data/output/final_report.docx")

from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)


from bsm_multi_agents.agents.pricing_validator_node import (
    pricing_validator_agent_node,
    pricing_validator_tool_node,
)

from bsm_multi_agents.agents.report_generator_node import (
    report_generator_agent_node,
)

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path,
    local_tool_paths=local_tool_paths,
    final_report_path=final_report_path,
)

state = pricing_calculator_agent_node(initial_state)
state = pricing_calculator_tool_node(state)
state = pricing_validator_agent_node(state)
state = pricing_validator_tool_node(state)
state = report_generator_agent_node(state)
state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/mcp/server.py',
 'local_tool_paths': ['/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/tools/my_add.py'],
 'final_report_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/output/final_report.docx',
 'messages': [SystemMessage(content='You are a quantitative calculator agent. You have access to tools specifically for Greeks calculation via an MCP server. Use the available tools to process the requested data. If you are confident, you can run all tools in parallel.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Input CSV File: /Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv\nOutput Directory: /Users/yifanli/Githu

# Graph

In [8]:
from langgraph.graph import StateGraph, END, START

from bsm_multi_agents.graph.state import WorkflowState
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)

from bsm_multi_agents.agents.pricing_validator_node import (
    pricing_validator_agent_node,
    pricing_validator_tool_node,
)

from bsm_multi_agents.agents.report_generator_node import (
    report_generator_agent_node,
)


In [9]:
def should_continue_for_pricing_calculator(state):
    messages = state["messages"]
    if not messages:
        return END
        
    last_msg = messages[-1]
    
    # 1. If it's an Agent message (AIMessage) with tool_calls -> Go to Tool
    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        return "pricing_calculator_tool"
    
    # 2. If it's a Tool Execution message (ToolMessage)
    #    Check for errors to decide if we should retry
    if isinstance(last_msg, ToolMessage):
         if last_msg.content.startswith("Error"):
             # OPTIONAL: Check iteration count to prevent infinite loop
             return "pricing_calculator_agent"
         
         # Success -> Finish
         return "pricing_validator_agent"
    
    # Default
    return "pricing_validator_agent"

def should_continue_for_pricing_validator(state):
    messages = state["messages"]
    if not messages:
        return END
        
    last_msg = messages[-1]
    
    # 1. If it's an Agent message (AIMessage) with tool_calls -> Go to Tool
    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        return "pricing_validator_tool"
    
    # 2. If it's a Tool Execution message (ToolMessage)
    #    Check for errors to decide if we should retry
    if isinstance(last_msg, ToolMessage):
         if last_msg.content.startswith("Error"):
             # OPTIONAL: Check iteration count to prevent infinite loop
             return "pricing_validator_agent"
         
         # Success -> Finish
         return "report_generator_agent"
    
    # Default
    return "report_generator_agent"

In [10]:
graph = StateGraph(WorkflowState)
graph.add_node("pricing_calculator_agent", pricing_calculator_agent_node)
graph.add_node("pricing_calculator_tool", pricing_calculator_tool_node)
graph.add_node("pricing_validator_agent", pricing_validator_agent_node)
graph.add_node("pricing_validator_tool", pricing_validator_tool_node)
graph.add_node("report_generator_agent", report_generator_agent_node)

graph.add_edge(START, "pricing_calculator_agent")
graph.add_edge("pricing_calculator_agent", "pricing_calculator_tool")
graph.add_conditional_edges(
    "pricing_calculator_tool",
    should_continue_for_pricing_calculator,
    {
        "pricing_calculator_agent": "pricing_calculator_agent", # Retry
        "pricing_validator_agent": "pricing_validator_agent" # Success
    }
)
graph.add_edge("pricing_validator_agent", "pricing_validator_tool")
graph.add_conditional_edges(
    "pricing_validator_tool",
    should_continue_for_pricing_validator,
    {
        "pricing_validator_agent": "pricing_validator_agent", # Retry
        "report_generator_agent": "report_generator_agent" # Success
    }
)
graph.add_edge("report_generator_agent", END)
app = graph.compile()

In [11]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
local_tool_paths = [os.path.join(project_path, "src/bsm_multi_agents/tools/my_add.py")]
final_report_path = os.path.join(project_path, "data/output/final_report.docx")

init_state = WorkflowState(
    csv_file_path=csv_file_path,
    output_dir=output_dir,
    server_path=server_path,
    local_tool_paths=local_tool_paths,
    final_report_path=final_report_path,
    # "remaining_steps": 10,
    # "messages": [HumanMessage(content=f"Load CSV from: {file_path}")],
)

final_state = app.invoke(
    init_state,
    config={"configurable": {"thread_id": "run-1"}}
)

In [13]:
final_state

{'messages': [SystemMessage(content='You are a quantitative calculator agent. You have access to tools specifically for Greeks calculation via an MCP server. Use the available tools to process the requested data. If you are confident, you can run all tools in parallel.', additional_kwargs={}, response_metadata={}, id='48e9a4a0-1671-4356-8145-c0ddba941b91'),
  HumanMessage(content='Input CSV File: /Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv\nOutput Directory: /Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache\n\nPlease calculate the Greeks for the options in the input CSV file. Save the results to the output directory. Ensure you call the calculation tools.', additional_kwargs={}, response_metadata={}, id='e6fd5356-4926-4669-ba04-1453950362c0'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3:8b', 'created_at': '2025-12-18T00:31:27.20362Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6216116